In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from numpy import random
from scipy.stats import binom
import scipy.stats as stats
from statsmodels.tsa.arima_process import ArmaProcess
from scipy.special import digamma,polygamma, gamma
from scipy.linalg import sqrtm
from tqdm import tqdm

In [ ]:
# the time-varying probability; return a vector lag 1 with exo variables version 
def threshold_logit(X_t,c=0):
    """
    Return the threshold logit transformation of X_t
    """
    X_t_trans = list(map(lambda x: min(max(c,x),1-c), X_t))
    X_t_trans[:] = list(map(lambda x: np.log(x/(1-x)), X_t_trans))
                         
    return X_t_trans
def ts_quantile_map(X, grid):
    """
    This function is used to get the quantiles of a given ts. 
    X: the input ts.
    grid: the percision of the measurement. We calculate the quantiles every 1/grid.
    """
    quan_list = [i/(grid+1) for i in range(1,grid+1)]

    x_quan_list = np.quantile(X, quan_list, interpolation = 'nearest')
    
    return quan_list, x_quan_list
                
def get_var_value(X, x_quan, length):
    """
    Given the ts X and the quantile value x_quan, the precision parameter 'length', 
    this function is used to calculate the cov.
    """
    
    bool_series = list(map(lambda z: int(z<=x_quan), X))
    cov = np.var(bool_series)
    for l in range(1,length+1):
        cov_l = 2*np.cov(bool_series[:-l], bool_series[l:])[0][1]
        cov+= cov_l
        
    return cov

def gaussian_quan_map(X, quan_list, x_quan_list, length):
    """
    This function is used to calculate the variance of the gaussian process, based on the given ts.
    X: the input ts that is used to calculate the cov matrix.
    quan_map: (quantile, the corresponding quantile value) based on  X.
    length: the number of i's included in the sum (used to control the precision of the cov matrix).
    """    
    cov_list = []
    x_quan_pre = -1
    cov = 0
    for i in range(len(x_quan_list)):
        x_quan = x_quan_list[i]   
        if x_quan == x_quan_pre:
            cov_list.append(cov)
        else:
            x_quan_pre = x_quan
            cov = get_var_value(X, x_quan, length)
            # avoid the situation that cov is negative 
            cov = np.max([0, cov])
            cov_list.append(cov)
    
    return list(zip(quan_list, x_quan_list, cov_list))

def get_quan_map(X, grid, length):
    quan_list, x_quan_list = ts_quantile_map(X, grid)
    quan_map = gaussian_quan_map(X, quan_list, x_quan_list, length)
    
    return quan_list, x_quan_list, quan_map
# ----------------------------------------------------------------------
def get_c_u(X, quan_map):
    """
    Given a ts X and the quantile map, this function is used to calculate all the c_u of which the u's are all the quantiles in the map.
    X: the input ts.
    quan_map: a list of tuple, each element is a 3-d tuple (quantile, x_quantile, the covariance of the guassian matrix).
    """
    c_u_list = []
    x_quan_pre = -1
    c_u = 0
    for i in range(len(quan_map)):
        x_quan = quan_map[i][1]
        if x_quan == x_quan_pre:
            c_u_list.append(c_u)
        else:
            x_quan_pre = x_quan
            c_u = np.mean(list(map(lambda z: int(z<=x_quan), X)))
            c_u_list.append(c_u)
    
    return c_u_list
def Dm(m,k,c_u_list_new, c_u_list_orgin):
    """
    This function is used to calculate the test vector Dm(s,k).
    m: the amount of original inputs.
    k: the amount of new inputs.
    c_u_list_new: the c_u list of the new ts, with all the quantiles.
    c_u_list_orgin: the c_u list of the original ts, with all the quantiles.
    """
    diff_list = list(map(lambda c1,c2: (c1-c2), c_u_list_new, c_u_list_orgin))   
    Dm_stat = list(map(lambda x: (k+m)*x/np.sqrt(m), diff_list))
    
    return Dm_stat

def max_test_stat(m,k,c_u_list_new, c_u_list_orgin):
    """
    This function is used to calculate the test statistic.
    m: the amount of original inputs.
    k: the amount of new inputs.
    c_u_list_new: the c_u list of the new ts, with all the quantiles.
    c_u_list_orgin: the c_u list of the original ts, with all the quantiles.
    """
    diff_list = list(map(lambda c1,c2: abs(c1-c2), c_u_list_new, c_u_list_orgin))
    max_diff = np.max(diff_list)

    test_stat = (k+m)*max_diff/np.sqrt(m)
    
    return test_stat

def get_gaussian(sgrid, N, quan_map, nIteration, randSeed):

    s_list = [i*N/sgrid+1 for i in range(1,sgrid+1)]
    max_value_list = []
    np.random.seed(randSeed)
    for _ in range(nIteration):
        max_value_u = 0
        for i in range(len(quan_map)):
            for s in s_list:
                cov = s*(s-1)*quan_map[i][2]
                sd = np.sqrt(cov)
                max_value_u = np.max([max_value_u,abs(np.random.normal(0, sd))])
        max_value_list.append(max_value_u) 
        
    return max_value_list

def get_cov_value(X, x_quan1, x_quan2, length):
    """
    Given the ts X and the quantile value x_quan, the precision parameter 'length', 
    this function is used to calculate the cov.
    """
    
    bool_series1 = list(map(lambda z: int(z<=x_quan1), X))
    bool_series2 = list(map(lambda z: int(z<=x_quan2), X))

    cov = np.cov(bool_series1, bool_series2)[0][1]
    for l in range(1,length+1):
        cov_l = np.cov(bool_series1[:-l], bool_series2[l:])[0][1]+np.cov(bool_series2[:-l], bool_series1[l:])[0][1]
        cov+= cov_l
        
    return cov
def get_cov_mat(X, x_quan_list, length):
    """
    Given the ts X and the quantile value x_quan, the precision parameter 'length', 
    this function is used to calculate the cov.
    """
    l = len(x_quan_list)
    cov_mat = []
    for x_quan1 in x_quan_list:
        for x_quan2 in x_quan_list:
            cov = get_cov_value(X, x_quan1, x_quan2, length)
            cov_mat.append(cov)
    cov_mat = np.reshape(cov_mat,(l,l))
    
        
    return cov_mat

In [ ]:
def geneWiener_1d(m,N,Sigma1,Sigma2):
    # m is the total of the initial observations
    # Nm is the upper bound of k
    # Sigma is the var-cov matrix of the wiener process
    # ------ first we generate the Wiener process free of t

    sd = np.sqrt(Sigma2)
    W2 = np.random.normal(0, sd)
    X = [] # store the generated normal r.v.
    for i in range(int(N*m)):
        x = np.random.normal(0, sd)
        
        X.append(x-1/np.sqrt(m)*W2)

    cumsum_X = np.cumsum(X)
    cumsum_X /= np.sqrt(m) # scaling
    
    return cumsum_X

In [ ]:
def geneWiener(m,N,Sigma1,Sigma2):
    # m is the total of the initial observations
    # Nm is the upper bound of k
    # Sigma is the var-cov matrix of the wiener process
    # ------ first we generate the Wiener process free of t
    l = len(Sigma1)
    mu_0 = [0]*l
    W2 = random.multivariate_normal(mu_0,Sigma2,1)
    X = [[] for i in range(l)] # store the generated normal r.v.
    for i in range(int(N*m)):
        x = random.multivariate_normal(mu_0,Sigma1,1)
        for k in range(l):
            X[k].append(x[0][k]-1/np.sqrt(m)*W2[0][k])

    cumsum_X = np.cumsum(X,axis = 1)
    cumsum_X /= np.sqrt(m) # scaling

    return cumsum_X

def rho(k,m0, gamma):
    rho_s = (1+k/m0)**(-1)*(k/(m0+k))**(-gamma)
    
    return rho_s

#-------- pack the generating process
def geneWiener_gamma(gamma_vector,Sigma,A,m0,N):
    wiener = geneWiener(m0,N,Sigma,Sigma)
    wiener = wiener.T 
    l = np.shape(wiener)[0]
    k = len(gamma_vector)
    wiener_wth_gamma = np.zeros((k,l))
    for gamma in gamma_vector:
        X = []
        for i in range(l):
            x = (1+(i+1)/m0)**(-2)*((i+1)/(m0+1+i))**(-2*gamma)*np.dot(np.dot(wiener[i],A),wiener[i])
            X.append(x)
        wiener_wth_gamma[gamma_vector.index(gamma)] = X
    return wiener_wth_gamma


In [ ]:
# given the true parameter vector beta, we can generate the first m-99 observations
def geneX_wth_exo(m,tau_true, phi_true, gamma_true,mu_0,W_t,randomSeed, functiontype='logit', c=0):
    """
    This function is used to generate betaAR(p) process based on the given true beta and true tau.

    m: the number of samples needed to be generated from the process.
    phi_true: parameter vector for x_t in the link function. p+1 dimensions.
    tau_true: dispersion parameter for the beta distribution.
    beta_true: the coefficient of x_t in the link function. scalar.
    mu_0: the expectation of the first p observations.
    W_t: the time series of exogenous variables. The length of X_t is equal to or greater than m-99.
    """
    X_t = []
    p = len(phi_true)-1
    q = len(gamma_true)
    alpha_0 = mu_0*tau_true
    beta_0  = (1-mu_0)*tau_true
        
    np.random.seed(randomSeed)
    x_t = list(np.random.beta(alpha_0,beta_0,p))
    X_t.append(x_t)
    X_t = sum(X_t,[])
    
    for i in range(m+p+100):
        X_tsub = X_t[i:(i+p)]
        W_tsub = W_t[i:(i+q)]
        if functiontype=='logit':
            hX_tsub = threshold_logit(X_tsub,c)
        if functiontype=='identity':
            hX_tsub = X_tsub
        mu_t = (1+np.exp(-phi_true[0]-np.inner(phi_true[1:],hX_tsub)-np.inner(W_tsub, gamma_true)))**(-1)
        alpha_t = mu_t*tau_true
        beta_t  = (1-mu_t)*tau_true
                
        x_t = np.random.beta(alpha_t,beta_t,1)
        
        X_t.extend(x_t)
        
    X_t = X_t[(100+p):]   
    W_t_used = W_t[(100+p):(100+m+p+q)]

    return X_t, W_t_used

def geneX_and_W(m,tau_true, phi_true, gamma_true,mu_0,ar_coef, ma_coef ,randomSeed, functiontype = 'logit', c=0):

    AR_object1 = ArmaProcess(ar_coef, ma_coef)
    simulated_data_1 = AR_object1.generate_sample(nsample=m+200)

    X_t_logit,W_t = geneX_wth_exo(m,tau_true,phi_true,gamma_true,mu_0,simulated_data_1,randomSeed, functiontype, c)
    
    return X_t_logit, W_t

# Under the H_0

In [ ]:
ar_coef, ma_coef = np.array([1, 0.1]), np.array([1])
tau_true, phi_true, gamma_true = 100, [0.5,0.1,0.2,0.2], [0.5]
N = 2
m = 50
n = (N+1)*m
grid = 20
length = 20
nIteration = 5000
gamma_vector = [0,0.25,0.4]

In [ ]:
def threshold_generation(tau_true, phi_true, gamma_true, ar_coef, ma_coef,\
                         m,N,grid, length, nIteration):
    
    y_list,_ = geneX_and_W(10000,tau_true, phi_true, gamma_true,0.5,ar_coef, ma_coef ,randomSeed = 2022)
    quan_list, x_quan_list, quan_map = get_quan_map(y_list, grid, 1)
    cov_mat = get_cov_mat(y_list, x_quan_list, length)

    A = np.linalg.inv(cov_mat)
    
    sup_list = []
    for _ in tqdm(range(nIteration)):   
        cumsum_list = geneWiener_gamma(gamma_vector,cov_mat,A,50,N)
        sup_cumsum = np.max(cumsum_list, axis = 1)
        sup_list.append(sup_cumsum)

    thres = np.quantile(sup_list, [0.9, 0.95, 0.975, 0.99], axis = 0, interpolation = 'nearest')
    
    return thres, quan_list, x_quan_list, quan_map, A

In [ ]:
def betaARsample_generation(tau_true, phi_true, gamma_true, ar_coef, ma_coef, A, \
           m,N,grid, randSeedstart, nYlist, quan_map):
    """
    The function generates W_t, X_t and calculate the test statistics.
    tau_true, phi_true, gamma_true: parameter sets for X_t.
    ar_coef, ma_coef: parameter sets for W_t.
    A: the p.d. matrix in the test statistic. 
    m: the number of initial observations.
    N: Nm is the total number of new observations.
    grid: the number of sample quantiles.
    nIteration: the number of iterations of threshold estimation.
    randSeedstart: the start point of the random seed in X_t simulation.
    nYlist: the number of simulated sequences.
    quan_map: the quantile, x, covariance list for the sequence.
    """

    test_s_list = []
    for randSeed in tqdm(range(randSeedstart, randSeedstart+nYlist)):
        y_list,_ = geneX_and_W((N+1)*m,tau_true, phi_true, gamma_true,0.5,ar_coef, ma_coef ,randSeed)
        y_list_origin = y_list[:m]

        test_s_max = np.zeros(len(gamma_vector))
        k = 1
        c_u_list_orgin = get_c_u(y_list_origin, quan_map)
        while (k<(N*m+1)):
            c_u_list_new = get_c_u(y_list[:(k+m)], quan_map)
            Dm_vector = Dm(m,k,c_u_list_new, c_u_list_orgin)

            for i in range(len(gamma_vector)):
                rho_s = rho(k,m, gamma_vector[i])
                test_s =  rho_s**2*np.dot(np.dot(Dm_vector,A),Dm_vector)
                test_s_max[i] = np.max([test_s_max[i], test_s])        
            k+=1
        test_s_list.append(test_s_max)
    test_s_array = np.array(test_s_list).T
    
    return test_s_array

In [ ]:
thres, quan_list, x_quan_list, quan_map, A = threshold_generation(tau_true, phi_true, gamma_true, ar_coef, ma_coef,\
                         m,N,grid, length, nIteration)

In [ ]:
m = 50
randSeedstart, nYlist = 0, 5000
test_s_array = betaARsample_generation(tau_true, phi_true, gamma_true, ar_coef, ma_coef, A,\
           m,N,grid, randSeedstart, nYlist, quan_map)

In [ ]:
# 5000
emp_alpha = np.zeros((4,3))
for i in range(4):
    for j in range(3):
        emp_alpha[i][j] = 1-sum(list(map(lambda x: int(x>=thres[i][j]), test_s_array[j])))/len(test_s_array[0])
emp_alpha

In [ ]:
# 1000
emp_alpha = np.zeros((4,3))
for i in range(4):
    for j in range(3):
        emp_alpha[i][j] = 1-sum(list(map(lambda x: int(x>=thres[i][j]), test_s_array[j])))/len(test_s_array[0])
emp_alpha

In [ ]:
m = 100
randSeedstart, nYlist = 200, 5000
test_s_array = betaARsample_generation(tau_true, phi_true, gamma_true, ar_coef, ma_coef, A,\
           m,N,grid, length, randSeedstart, nYlist, quan_map)

In [ ]:
# 5000
emp_alpha = np.zeros((4,3))
for i in range(4):
    for j in range(3):
        emp_alpha[i][j] = 1-sum(list(map(lambda x: int(x>=thres[i][j]), test_s_array[j])))/len(test_s_array[0])
emp_alpha

In [ ]:
m = 150
randSeedstart, nYlist = 1231, 5000
test_s_array = betaARsample_generation(tau_true, phi_true, gamma_true, ar_coef, ma_coef, A, \
           m,N,grid, length, randSeedstart, nYlist, quan_map)

In [ ]:
# 5000
emp_alpha = np.zeros((4,3))
for i in range(4):
    for j in range(3):
        emp_alpha[i][j] = 1-sum(list(map(lambda x: int(x>=thres[i][j]), test_s_array[j])))/len(test_s_array[0])
emp_alpha

In [ ]:
# 1000
emp_alpha = np.zeros((4,3))
for i in range(4):
    for j in range(3):
        emp_alpha[i][j] = 1-sum(list(map(lambda x: int(x>=thres[i][j]), test_s_array[j])))/len(test_s_array[0])
emp_alpha

In [ ]:
A = np.eye(20)/20
m = 500
randSeedstart, nYlist = 0, 1000
test_s_array = betaARsample_generation(tau_true, phi_true, gamma_true, ar_coef, ma_coef, A,\
           m,N,grid, randSeedstart, nYlist, quan_map)

In [ ]:
emp_alpha = np.zeros((4,3))
for i in range(4):
    for j in range(3):
        emp_alpha[i][j] = 1-sum(list(map(lambda x: int(x>=thres[i][j]), test_s_array[j])))/len(test_s_array[0])
emp_alpha

In [ ]:
1-emp_alpha